CS4001/4042 Assignment 1, Part B, Q4
---

Model degradation is a common issue faced when deploying machine learning models (including neural networks) in the real world. New data points could exhibit a different pattern from older data points due to factors such as changes in government policy or market sentiments. For instance, housing prices in Singapore have been increasing and the Singapore government has introduced 3 rounds of cooling measures over the past years (16 December 2021, 30 September 2022, 27 April 2023).

In such situations, the distribution of the new data points could differ from the original data distribution which the models were trained on. Recall that machine learning models often work with the assumption that the test distribution should be similar to train distribution. When this assumption is violated, model performance will be adversely impacted.  In the last part of this assignment, we will investigate to what extent model degradation has occurred.




---



---



Your co-investigators used a linear regression model to rapidly test out several combinations of train/test splits and shared with you their findings in a brief report attached in Appendix A below. You wish to investigate whether your deep learning model corroborates with their findings.

In [ ]:
!pip install alibi-detect

In [11]:
SEED = 42

import os
import torch

import random
random.seed(SEED)

import numpy as np
np.random.seed(SEED)

import pandas as pd
from sklearn.metrics import r2_score
from pytorch_tabular import TabularModel

from alibi_detect.cd import TabularDrift

> Evaluate your model from B1 on data from year 2022 and report the test R2.

R2 Score: 0.577

In [13]:
df = pd.read_csv('hdb_price_prediction.csv')

df_eval_2022 = df[df['year'] == 2022]

tabular_model = TabularModel.load_from_checkpoint("hdb_regression")
pred_2022 = tabular_model.predict(df_eval_2022)

r2_score_2022 = r2_score(df_eval_2022['resale_price'], pred_2022['resale_price_prediction'])

print("R2 Score:", r2_score_2022)

2023-09-25 00:38:07,799 - {pytorch_tabular.tabular_model:129} - INFO - Experiment Tracking is turned off
2023-09-25 00:38:07,811 - {pytorch_tabular.tabular_model:268} - INFO - Preparing the Trainer
c:\Users\micha\miniconda3\envs\cz4042_nn_alt\lib\site-packages\pytorch_lightning\trainer\connectors\accelerator_connector.py:589: LightningDeprecationWarning: The Trainer argument `auto_select_gpus` has been deprecated in v1.9.0 and will be removed in v2.0.0. Please use the function `pytorch_lightning.accelerators.find_usable_cuda_devices` instead.
  rank_zero_deprecation(
Trainer already configured with model summary callbacks: [<class 'pytorch_lightning.callbacks.rich_model_summary.RichModelSummary'>]. Skipping setting a default `ModelSummary` callback.
GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


Output()

R2 Score: 0.5771082349260532


> Evaluate your model from B1 on data from year 2023 and report the test R2.

In [14]:
# TODO: Enter your code here
df = pd.read_csv('hdb_price_prediction.csv')

df_eval_2023 = df[df['year'] == 2023]

tabular_model = TabularModel.load_from_checkpoint("hdb_regression")
pred_2023 = tabular_model.predict(df_eval_2023)

r2_score_2023 = r2_score(df_eval_2023['resale_price'], pred_2023['resale_price_prediction'])

print("R2 Score:", r2_score_2023)

2023-09-25 00:38:39,073 - {pytorch_tabular.tabular_model:129} - INFO - Experiment Tracking is turned off
2023-09-25 00:38:39,084 - {pytorch_tabular.tabular_model:268} - INFO - Preparing the Trainer
c:\Users\micha\miniconda3\envs\cz4042_nn_alt\lib\site-packages\pytorch_lightning\trainer\connectors\accelerator_connector.py:589: LightningDeprecationWarning: The Trainer argument `auto_select_gpus` has been deprecated in v1.9.0 and will be removed in v2.0.0. Please use the function `pytorch_lightning.accelerators.find_usable_cuda_devices` instead.
  rank_zero_deprecation(
Trainer already configured with model summary callbacks: [<class 'pytorch_lightning.callbacks.rich_model_summary.RichModelSummary'>]. Skipping setting a default `ModelSummary` callback.
GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


Output()

R2 Score: 0.32164634214074306


> Did model degradation occur for the deep learning model?


Yes model degredation did occur. As the model was train on <= 2019 data, there might have been real world conditions affecting the data features of the model in the year 2023.



---



---



Model degradation could be caused by [various data distribution shifts](https://huyenchip.com/2022/02/07/data-distribution-shifts-and-monitoring.html#data-shift-types): covariate shift (features), label shift and/or concept drift (altered relationship between features and labels).
There are various conflicting terminologies in the [literature](https://www.sciencedirect.com/science/article/pii/S0950705122002854#tbl1). Let’s stick to this reference for this assignment.

> Using the **Alibi Detect** library, apply the **TabularDrift** function with the training data (year 2019 and before) used as the reference and **detect which features have drifted** in the 2023 test dataset. Before running the statistical tests, ensure you **sample 1000 data points** each from the train and test data. Do not use the whole train/test data. (Hint: use this example as a guide https://docs.seldon.io/projects/alibi-detect/en/stable/examples/cd_chi2ks_adult.html)


In [52]:
# TODO: Enter your code here
num_col_names = ['dist_to_nearest_stn', 'dist_to_dhoby', 'degree_centrality', 'eigenvector_centrality', 'remaining_lease_years', 'floor_area_sqm']
cat_col_names = ['month', 'town', 'flat_model_type', 'storey_range']

df_reference = df[df['year'] <= 2019]
df_test = df[df['year'] == 2023]

y_df_reference = df_reference['resale_price']
y_df_test = df_test['resale_price']

# df_reference = df_reference.drop(columns=['year', 'full_address', 'nearest_stn', 'resale_price'])
# df_test = df_test.drop(columns=['year', 'full_address', 'nearest_stn', 'resale_price'])

n_ref = 1000
n_test = 1000
df_reference = df_reference[:n_ref]
df_test = df_test[:n_test]

categories_per_feature = {df.columns.get_loc(f): None for f in cat_col_names}

cd = TabularDrift(df_reference.values, p_val=.05, categories_per_feature=categories_per_feature)

preds = cd.predict(df_test.values)
labels = ['No!', 'Yes!']
print('Drift? {}'.format(labels[preds['data']['is_drift']]))

Drift? Yes!


In [55]:
feature_names = df_test.columns.values
fpreds = cd.predict(df_test.values, drift_type='feature')

for f in range(cd.n_features):
    stat = 'Chi2' if f in list(categories_per_feature.keys()) else 'K-S'
    fname = feature_names[f]
    is_drift = fpreds['data']['is_drift'][f]
    stat_val, p_val = fpreds['data']['distance'][f], fpreds['data']['p_val'][f]
    print(f'{fname} -- Drift? {labels[is_drift]} -- {stat} {stat_val:.3f} -- p-value {p_val:.3f}')

month -- Drift? No! -- Chi2 0.000 -- p-value 1.000
year -- Drift? Yes! -- K-S 1.000 -- p-value 0.000
town -- Drift? Yes! -- Chi2 667.474 -- p-value 0.000
full_address -- Drift? Yes! -- K-S 0.061 -- p-value 0.046
nearest_stn -- Drift? Yes! -- K-S 0.251 -- p-value 0.000
dist_to_nearest_stn -- Drift? No! -- K-S 0.055 -- p-value 0.094
dist_to_dhoby -- Drift? Yes! -- K-S 0.218 -- p-value 0.000
degree_centrality -- Drift? No! -- K-S 0.029 -- p-value 0.783
eigenvector_centrality -- Drift? Yes! -- K-S 0.195 -- p-value 0.000
flat_model_type -- Drift? Yes! -- Chi2 77.586 -- p-value 0.000
remaining_lease_years -- Drift? Yes! -- K-S 0.271 -- p-value 0.000
floor_area_sqm -- Drift? Yes! -- K-S 0.134 -- p-value 0.000
storey_range -- Drift? Yes! -- Chi2 38.800 -- p-value 0.001
resale_price -- Drift? Yes! -- K-S 0.326 -- p-value 0.000


> Assuming that the flurry of housing measures have made an impact on the relationship between all the features and resale_price (i.e. P(Y|X) changes), which type of data distribution shift possibly led to model degradation?


\# TODO: \<Enter your answer here\>

> From your analysis via TabularDrift, which features contribute to this shift?


\# TODO: \<Enter your answer here\>

> Suggest 1 way to address model degradation and implement it, showing improved test R2 for year 2023.


\# TODO: \<Enter your answer here\>

In [ ]:
# TODO: Enter your code here

### Appendix A



Here are our results from a linear regression model. We used StandardScaler for continuous variables and OneHotEncoder for categorical variables.

While 2021 data can be predicted well, test R2 dropped rapidly for 2022 and 2023.

| Training set | Test set | Test R2 |
|--------------|----------|---------|
| Year <= 2020 | 2021     | 0.76    |
| Year <= 2020 | **2022**     | 0.41    |
| Year <= 2020 | **2023**     | **0.10**   |



Similarly, a model trained on 2017 data can predict 2018-2021 well (with slight degradation in performance for 2021), but drops drastically in 2022 and 2023.

| Training set | Test set | Test R2 |
|--------------|----------|---------|
| 2017         | 2018     | 0.90    |
|              | 2019     | 0.89    |
|              | 2020     | 0.87    |
|              | 2021     | 0.72    |
|              | **2022**     | **0.37**    |
|              | **2023**     | **0.09**    |

With the test set fixed at year 2021, training on data from 2017-2020 still works well on the test data, with minimal degradation. Training sets closer to year 2021 generally do better.

| Training set | Test set | Test R2 |
|--------------|----------|---------|
| 2020         | 2021     | 0.81    |
| 2019         | 2021     | 0.75    |
| 2018         | 2021     | 0.73    |
| 2017         | 2021     | 0.72    |